## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.24,94,65,5.10,broken clouds
1,1,Port Macquarie,AU,-31.4333,152.9167,68.85,74,98,8.05,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,42.12,81,92,11.95,light rain
3,3,Shimoda,JP,34.6667,138.9500,73.11,62,100,7.63,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,77.63,84,71,19.95,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Rikitea,PF,-23.1203,-134.9692,77.63,84,71,19.95,light rain
6,6,Touros,BR,-5.1989,-35.4608,76.12,93,99,11.01,light rain
9,9,Hilo,US,19.7297,-155.0900,78.28,93,100,5.01,mist
10,10,Aswan,EG,24.0934,32.9070,78.10,29,0,11.50,clear sky
11,11,Avarua,CK,-21.2078,-159.7750,82.45,74,20,9.22,few clouds
12,12,Kapaa,US,22.0752,-159.3190,82.38,77,40,14.97,moderate rain
13,13,Sao Joao Da Barra,BR,-21.6403,-41.0511,76.86,88,71,16.11,broken clouds
15,15,Souillac,MU,-20.5167,57.5167,75.83,83,72,18.61,light rain
17,17,Srivardhan,IN,18.0333,73.0167,81.84,76,0,7.34,clear sky
18,18,Mitsamiouli,KM,-11.3847,43.2844,79.84,73,74,7.09,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                222
City                   222
Country                222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,77.63,light rain,-23.1203,-134.9692,
6,Touros,BR,76.12,light rain,-5.1989,-35.4608,
9,Hilo,US,78.28,mist,19.7297,-155.0900,
10,Aswan,EG,78.10,clear sky,24.0934,32.9070,
11,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
12,Kapaa,US,82.38,moderate rain,22.0752,-159.3190,
13,Sao Joao Da Barra,BR,76.86,broken clouds,-21.6403,-41.0511,
15,Souillac,MU,75.83,light rain,-20.5167,57.5167,
17,Srivardhan,IN,81.84,clear sky,18.0333,73.0167,
18,Mitsamiouli,KM,79.84,broken clouds,-11.3847,43.2844,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
 # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



KeyError: 'Weather Description'

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig